In [1]:
import logging
# Set in to avoid following issue:
#  No handlers could be found for logger "kazoo.client"
logging.basicConfig()

from kazoo.client import KazooClient
zk = KazooClient()
zk.start()

In [2]:
# Same as `ls /`
zk.get_children("/")

['zookeeper']

In [3]:
zk.get_children("/zookeeper"), zk.get_children("/zookeeper/quota")

(['quota'], [])

In [4]:
zk.create(path="/mynode", value=b"first_version")

'/mynode'

In [5]:
zk.get_children("/")

['mynode', 'zookeeper']

In [6]:
zk.get('/mynode')  # Returns tuple (Value, Metadata), same as get /mynode && stat /mynode

(b'first_version',
 ZnodeStat(czxid=2, mzxid=2, ctime=1666874586670, mtime=1666874586670, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=2))

In [7]:
zk.set('/mynode', value=b"second_version")

ZnodeStat(czxid=2, mzxid=3, ctime=1666874586670, mtime=1666874610271, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=14, numChildren=0, pzxid=2)

In [8]:
zk.get('/mynode') 

(b'second_version',
 ZnodeStat(czxid=2, mzxid=3, ctime=1666874586670, mtime=1666874610271, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=14, numChildren=0, pzxid=2))

## Create two sequential nodes

In [9]:
# Same as create -s /mynode/child 'data'
zk.create('/mynode/child', value=b'im_sequential', sequence=True)
zk.create('/mynode/child', value=b'im_sequential_too', sequence=True)

'/mynode/child0000000001'

In [11]:
zk.get_children('/mynode')

['child0000000000', 'child0000000001']

## Simultanious connections and Ephemeral nodes

In [12]:
zk1, zk2 = KazooClient(), KazooClient()
zk1.start()
zk2.start()

In [13]:
zk.create('/mygroup', b'top_node')

'/mygroup'

In [14]:
zk1.create('/mygroup/grue', b'im_grue', ephemeral=True)
zk2.create('/mygroup/bleen', b'im_bleen', ephemeral=True)

'/mygroup/bleen'

In [15]:
zk.get_children('/mygroup')

['grue', 'bleen']

In [16]:
 zk1.get('/mygroup/bleen')

(b'im_bleen',
 ZnodeStat(czxid=12, mzxid=12, ctime=1666874883834, mtime=1666874883834, version=0, cversion=0, aversion=0, ephemeralOwner=72057636928880644, dataLength=8, numChildren=0, pzxid=12))

In [17]:
zk2.stop()

In [18]:
zk.get_children('/mygroup')

['grue']

In [19]:
zk1.get('/mygroup/bleen')

NoNodeError: 

In [20]:
zk1.stop(), zk2.stop()

(None, None)

In [21]:
 zk.delete('/mygroup', recursive=True)

##  Distributed Application Configuration

In [22]:
zk.create('/myconfig', b'sheep_count=1')

'/myconfig'

In [23]:
zk1 = KazooClient()
zk1.start()

In [24]:
def watch_node(state):
    print(state)

In [25]:
zk1.get('/myconfig', watch=watch_node)

(b'sheep_count=1',
 ZnodeStat(czxid=16, mzxid=16, ctime=1666874889761, mtime=1666874889761, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=16))

In [26]:
zk.set('/myconfig', b'sheep_count=2')

WatchedEvent(type='CHANGED', state='CONNECTED', path='/myconfig')


ZnodeStat(czxid=16, mzxid=18, ctime=1666874889761, mtime=1666874891074, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=16)

In [27]:
zk.set('/myconfig', b'sheep_count=3')  # Trigger (watcher) is resetted 

ZnodeStat(czxid=16, mzxid=19, ctime=1666874889761, mtime=1666874891983, version=2, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=16)

## cli outputs

```
# echo conf | nc localhost 2181
clientPort=2181
dataDir=/data/version-2
dataLogDir=/datalog/version-2
tickTime=2000
maxClientCnxns=60
minSessionTimeout=4000
maxSessionTimeout=40000
serverId=0
# echo stat | nc localhost 2181
Zookeeper version: 3.4.14-4c25d480e66aadd371de8bd2fd8da255ac140bcf, built on 03/06/2019 16:18 GMT
Clients:
 /127.0.0.1:49804[1](queued=0,recved=42,sent=42)
 /127.0.0.1:49798[1](queued=0,recved=43,sent=43)
 /127.0.0.1:49796[1](queued=0,recved=87,sent=87)
 /127.0.0.1:49800[1](queued=0,recved=43,sent=43)
 /127.0.0.1:49802[1](queued=0,recved=42,sent=42)
 /127.0.0.1:49810[0](queued=0,recved=1,sent=0)

Latency min/avg/max: 0/0/19
Received: 259
Sent: 258
Connections: 6
Outstanding: 0
Zxid: 0xa
Mode: standalone
Node count: 8
# echo ruok | nc localhost 2181
imok# echo cons | nc localhost 2181
 /127.0.0.1:49804[1](queued=0,recved=51,sent=51,sid=0x1000009fc800004,lop=PING,est=1666874662328,to=10000,lcxid=0x0,lzxid=0xa,lresp=1219435,llat=0,minlat=0,avglat=0,maxlat=2)
 /127.0.0.1:49798[1](queued=0,recved=52,sent=52,sid=0x1000009fc800001,lop=PING,est=1666874659941,to=10000,lcxid=0x0,lzxid=0xa,lresp=1218612,llat=0,minlat=0,avglat=0,maxlat=3)
 /127.0.0.1:49796[1](queued=0,recved=96,sent=96,sid=0x1000009fc800000,lop=PING,est=1666874548802,to=10000,lcxid=0xc,lzxid=0xa,lresp=1219596,llat=1,minlat=0,avglat=1,maxlat=19)
 /127.0.0.1:49800[1](queued=0,recved=52,sent=52,sid=0x1000009fc800002,lop=PING,est=1666874659948,to=10000,lcxid=0x0,lzxid=0xa,lresp=1220369,llat=1,minlat=0,avglat=0,maxlat=2)
 /127.0.0.1:49802[1](queued=0,recved=51,sent=51,sid=0x1000009fc800003,lop=PING,est=1666874662310,to=10000,lcxid=0x0,lzxid=0xa,lresp=1219568,llat=1,minlat=0,avglat=0,maxlat=13)
 /127.0.0.1:49814[0](queued=0,recved=1,sent=0)
```

### Before and after we connect watched - we can see a new connection is added

```
# echo dump | nc localhost 2181
SessionTracker dump:
Session Sets (6):
0 expire at Thu Jan 01 00:21:10 UTC 1970:
0 expire at Thu Jan 01 00:21:12 UTC 1970:
0 expire at Thu Jan 01 00:21:14 UTC 1970:
0 expire at Thu Jan 01 00:21:16 UTC 1970:
4 expire at Thu Jan 01 00:21:18 UTC 1970:
        0x1000009fc800003
        0x1000009fc800002
        0x1000009fc800000
        0x1000009fc800004
1 expire at Thu Jan 01 00:21:20 UTC 1970:
        0x1000009fc800001
ephemeral nodes dump:
Sessions with Ephemerals (0):
# echo wchs | nc localhost 2181
0 connections watching 0 paths
Total watches:0
# echo wchs | nc localhost 2181
1 connections watching 0 paths
Total watches:0
#
```